In [ ]:
import os

import pandas as pd
import numpy as np

from zoobot.shared import label_metadata, schemas

In [ ]:
# final version of https://github.com/mwalmsley/curation-datasets/blob/main/notebooks/rename_gz2_catalog.ipynb

In [ ]:
# TODO review where exactly this comes from - hart16?
master_df = pd.read_parquet('/run/media/walml/beta/galaxy_zoo/gz2/subjects/image_master_catalog.parquet')
"""
original    209294
extra        21844
stripe82      8557
"""
master_df = master_df.query('sample == "original"')
len(master_df)

In [ ]:
renamer = {
    't01_smooth_or_features_a01_smooth': 'smooth-or-featured_smooth',
    't01_smooth_or_features_a02_features_or_disk': 'smooth-or-featured_featured-or-disk',
    't01_smooth_or_features_a03_star_or_artifact': 'smooth-or-featured_artifact',
    't02_edgeon_a04_yes': 'disk-edge-on_yes',
    't02_edgeon_a05_no': 'disk-edge-on_no',
    't03_bar_a06_bar': 'bar_yes',
    't03_bar_a07_no_bar': 'bar_no',
    't04_spiral_a08_spiral': 'has-spiral-arms_yes',
    't04_spiral_a09_no_spiral': 'has-spiral-arms_no',
    't05_bulge_prominence_a10_no_bulge': 'bulge-size_no',
    't05_bulge_prominence_a11_just_noticeable': 'bulge-size_just-noticeable',
    't05_bulge_prominence_a12_obvious': 'bulge-size_obvious',
    't05_bulge_prominence_a13_dominant': 'bulge-size_dominant',
    't06_odd_a14_yes': 'something-odd_yes',
    't06_odd_a15_no': 'something-odd_no',
    't07_rounded_a16_completely_round': 'how-rounded_round',
    't07_rounded_a17_in_between': 'how-rounded_in-between',
    't07_rounded_a18_cigar_shaped': 'how-rounded_cigar',
    't09_bulge_shape_a25_rounded': 'bulge-shape_round',
    't09_bulge_shape_a26_boxy': 'bulge-shape_boxy',
    't09_bulge_shape_a27_no_bulge': 'bulge-shape_no-bulge',
    't10_arms_winding_a28_tight': 'spiral-winding_tight',
    't10_arms_winding_a29_medium': 'spiral-winding_medium',
    't10_arms_winding_a30_loose': 'spiral-winding_loose',
    't11_arms_number_a31_1': 'spiral-arm-count_1',
    't11_arms_number_a32_2': 'spiral-arm-count_2',
    't11_arms_number_a33_3': 'spiral-arm-count_3',
    't11_arms_number_a34_4': 'spiral-arm-count_4',
    't11_arms_number_a36_more_than_4': 'spiral-arm-count_more-than-4',
    't11_arms_number_a37_cant_tell': 'spiral-arm-count_cant-tell'
}

renamer_with_count = dict([(key + '_count', value) for key, value in renamer.items()])

# copy the key id columns
df = master_df[['dr7objid', 'png_loc']]

# copy all label columns using renamer
for key, value in renamer_with_count.items():
    df.loc[:, value] = master_df[key]

df


In [ ]:

# now the schema object will work, I can calculate fractions and totals

question_answer_pairs = label_metadata.gz2_pairs
dependencies = label_metadata.gz2_and_decals_dependencies
schema = schemas.Schema(question_answer_pairs, dependencies)
for question in schema.questions:
    df[question.text + '_total-votes'] = df[[a.text for a in question.answers]].sum(axis=1)

    for answer in question.answers:
        df[answer.text + '_fraction'] = df[answer.text] / df[question.text + '_total-votes']





In [ ]:
# fix paths
df['file_loc'] = df['png_loc'].str.replace('/raid/scratch/walml', '/run/media/walml/beta')
df['file_loc'][0]


In [ ]:
# https://github.com/mwalmsley/curation-datasets/blob/main/downloadable_catalog.ipynb
# TODO merge further path changes and mapping to asset_id, unclear why I have both?

In [ ]:

# df['file_exists'] = df['file_loc'].apply(os.path.isfile)
# df['file_exists'].value_counts()

# df[~df['file_exists']].squeeze()['png_loc']
# '/raid/scratch/walml/galaxy_zoo/gz2/png/587736/587736915146703138.png'
# literally 1 image missing...

df = df[df['dr7objid'] != 587736915146703138]
# label_df['file_exists'].value_counts()

In [ ]:
# TODO merge with NSA here


In [ ]:

df.to_parquet('/run/media/walml/beta/galaxy_zoo/gz2/subjects/gz2_catalog_with_modern_schema.parquet', index=False)